<a href="https://colab.research.google.com/github/weqsful/collabfiles/blob/main/%D0%91%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82_3_%D0%9B%D0%B8%D0%B4%D0%B6%D0%B8%D0%B3%D0%BE%D1%80%D1%8F%D0%B5%D0%B2%D0%B0_%D0%94_%D0%90_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Итак, перед тем, как браться за статистику, нужно:

**1. Прочесть исходный файл и превратить его в структуру данных**

К заданию прилагается файл в формате csv, где все значения разделены запятыми. Это наши исходные данные. Чтобы применить к ним все возможности языка Python и библиотеки Pandas, надо импортировать эту библиотеку и сохранить её в переменной. По сокращённому названию панельных данных (panel data), с которых начиналась Pandas, эту переменную принято называть pd:

**import pandas as pd**

Для чтения csv-файла в библиотеке Pandas есть готовая функция — метод **read_csv()**. Как и все методы, он вызывается записью через точку после имени своего объекта. В скобках указывается аргумент (параметр) метода. У read_csv() это имя файла с данными. Прочтение превращает файл в структуру данных DataFrame. Имя переменной, в которой эта структура данных сохраняется, чаще всего df либо отражает тематику данных:

**df = pd.read_csv('music_log.csv')**

**2. Посмотреть на данные**

Вывести на экран таблицу и оценить данные:

print(df)
Как правило, таблица очень велика. Практичнее запросить определённое количество первых строк, методом head().

**3. Оценить качество предподготовки**

Нужно убедиться в том, что данные прошли предподготовку. По крайней мере, не должно быть пропусков и повторов. Пропущенные и неопределённые значения выявляет метод **isna()**, а суммарное количество таких значений — метод **sum()**.

**print(df.isna().sum())**


Повторяющиеся строки — дубликаты — выявляются методом **duplicated()** и подсчитываются тем же sum():

**print(df.duplicated().sum())**
Если возвращаются нули, всё хорошо — данные пригодны для исследования.


## Задача

1. Загрузите таблицу **exoplanet_catalog**, содержащую данные о планетах за пределами Солнечной системы, или экзопланетах. Сформируйте из данной таблицы новый датасет под назанием **exoplanet**, коорая будет содердать стоблцы name, mass, radius, discovered. Проверьте, чтобы название всех столбцов было корректным, при необходмости измените названия.

2. Удалите из получившейся таблица строки с нулевыми значениями.

3. Убедитесь, что таблица не содержит ни пропущенных значений, ни дубликатов.

In [ ]:
import pandas as pd
exoplanet_catalog = pd.read_csv('exoplanet_catalog.csv')
print(exoplanet_catalog)
exoplanet = exoplanet_catalog[['# name', 'mass', 'radius', 'discovered']]
exoplanet.columns = ['Название', 'Масса', 'Радиус', 'Дата открытия']

exoplanet = exoplanet[~exoplanet.isnull().any(axis=1)]

print("Количество пропущенных значений:", exoplanet.isna().sum())
print("Количество дубликатов:", exoplanet.duplicated().sum())
print(exoplanet)

         # name planet_status   mass  mass_error_min  mass_error_max  \
0      11 Com b     Confirmed    NaN             NaN             NaN   
1      11 Oph b     Confirmed  21.00            3.00            3.00   
2      11 UMi b     Confirmed    NaN             NaN             NaN   
3      14 And b     Confirmed    NaN             NaN             NaN   
4      14 Her b     Confirmed    NaN             NaN             NaN   
...         ...           ...    ...             ...             ...   
4791  ups And b     Confirmed   0.62            0.09             inf   
4792  ups And c     Confirmed   9.10            2.93            5.04   
4793  ups And d     Confirmed  23.58            2.29            2.93   
4794  ups And e     Confirmed    NaN             NaN             NaN   
4795  zet Del B     Confirmed  40.00            5.00           15.00   

      mass_sini  mass_sini_error_min  mass_sini_error_max  radius  \
0        19.400                1.500                1.500     NaN 

# Группировка данных

Слово «анализ» означает разбор, рассмотрение с разных сторон. Анализ данных начинают с разделения их на группы по какому-нибудь признаку. Эта операция называется группировка данных. Она помогает изучить материал более подробно, чтобы затем перейти к поиску взаимосвязей между отдельными группами.


Группировка оправданна, если данные чётко делятся по значимому признаку, а полученные группы близки к теме задачи. Например, когда есть данные обо всех покупках в супермаркете, можно смело заниматься группировкой. Так можно установить время наплыва покупателей и решить проблему пиковых нагрузок. Или посчитать средний чек — обычно для магазинов это ключевая метрика.


Стадии группировки хорошо укладываются в словесную формулу **split-apply-combine**:

•	разделить, **split** — разбиение на группы по определённому критерию;

•	применить, **apply** — применение какого-либо метода к каждой группе в отдельности, например, подсчёт численности группы методом count() или суммирование вызовом sum();

•	объединить, **combine** — сведение результатов в новую структуру данных, в зависимости от условий разделения и выполнения метода это бывает DataFrame и Series.

В библиотеке Pandas есть отличные инструменты группировки. Рассмотрим обращение с ними на примере анализа данных о планетах за пределами Солнечной системы, или экзопланетах. Орбитальные обсерватории засекли уже тысячи таких небесных тел. Их выявляют на снимках космических телескопов наши коллеги, аналитики данных. Поищем среди экзопланет похожие на Землю. Возможно, это наши будущие колонии, или там уже обитают разумные существа, с которыми однажды предстоит установить контакт.

DataFrame с данными по нескольким тысячам экзопланет сохранён в переменной **exoplanet**. Посмотрим на первые 30 строк таблицы:


In [ ]:
import pandas as pd
exoplanet_catalog = pd.read_csv('exoplanet_catalog.csv')
exoplanet = exoplanet_catalog[['# name', 'mass', 'radius', 'discovered']]
exoplanet.columns = ['Название', 'Масса', 'Радиус', 'Дата открытия']

print(exoplanet.head(30))


                   Название  Масса    Радиус  Дата открытия
0                  11 Com b    NaN       NaN         2008.0
1                  11 Oph b   21.0       NaN         2007.0
2                  11 UMi b    NaN       NaN         2009.0
3                  14 And b    NaN       NaN         2008.0
4                  14 Her b    NaN       NaN         2002.0
5                  14 Her c    5.8       NaN         2006.0
6                16 Cyg B b    NaN       NaN         1996.0
7                  17 Sco b    NaN       NaN         2020.0
8                  18 Del b    NaN       NaN         2008.0
9              1I/'Oumuamua    NaN  0.000002         2017.0
10              1RXS 1609 b   14.0  1.700000         2008.0
11  1RXS J235133.3+312720 b   32.0       NaN         2012.0
12           1SWASP J1407 b    NaN       NaN         2012.0
13                 24 Boo b    NaN       NaN         2018.0
14                 24 Sex b    NaN       NaN         2010.0
15                 24 Sex c    NaN      

**Документация**

Столбцы:

•	name: название экзопланеты;

•	mass: масса в массах планеты Юпитер;

•	radius: радиус, пересчитанный в радиусах Земли;

•	discovered: год открытия экзопланеты.

*Источник: каталог экзопланет на портале exoplanet.eu*

На картинке изображен принцип **split-apply-combine** для таблицы с экзопланетами. Посмотрим, как вообще идут дела с поиском экзопланет. Сначала данные делят по группам, где каждая группа — это год. Потом метод **count()** подсчитывает численность каждой группы. В итоге получаем новую структуру данных с группами, где каждая содержит год и число открытых за этот год экзопланет.

![image.png](attachment:image.png)

В Рandas для группировки данных есть метод **groupby()**. Он принимает как аргумент название столбца, по которому нужно группировать. В случае с делением экзопланет по годам открытия:


**print(exoplanet.groupby('discovered'))**

**<pandas.core.groupby.DataFrameGroupBy object at 0x7fc1e1ca3400>**


Применение метода **groupby()** к объекту типа DataFrame приводит к созданию объекта особого типа — **DataFrameGroupBy**. Это сгруппированные данные. Если применить к ним какой-нибудь метод Pandas, они станут новой структурой данных типа **DataFrame** или **Series**.
Подсчитаем сгруппированные по годам экзопланеты методом **count()**:


In [ ]:
import pandas as pd
exoplanet = pd.read_csv('exoplanet_catalog.csv')

grouped_exoplanets = exoplanet.groupby('discovered')

count_exoplanets = grouped_exoplanets.count()

print(count_exoplanets)


            # name  planet_status  mass  mass_error_min  mass_error_max  \
discovered                                                                
1988.0           1              1     1               0               0   
1992.0           3              3     3               1               1   
1995.0           3              3     3               2               2   
1996.0           7              7     4               3               3   
1998.0           8              8     1               1               1   
1999.0          13             13     6               6               6   
2000.0          23             23    11               9               9   
2001.0          15             15     6               5               5   
2002.0          33             33     8               8               8   
2003.0          26             26     6               4               4   
2004.0          35             35    13              11              11   
2005.0          35       

Если нужно сравнить наблюдения по одному показателю, метод применяют к **DataFrameGroupBy** с указанием на один столбец. Нас в первую очередь интересует радиус экзопланет: мы ищем другую Землю. Давайте получим таблицу с единственным столбцом 'radius':

**exo_number = exoplanet.groupby('discovered')['radius'].count()**

**print(exo_number)**

In [ ]:
import pandas as pd
exoplanet = pd.read_csv('exoplanet_catalog.csv')

exo_number = exoplanet.groupby('discovered')['radius'].count()

print(exo_number)

discovered
1988.0       0
1992.0       0
1995.0       1
1996.0       1
1998.0       0
1999.0       2
2000.0       5
2001.0       1
2002.0       4
2003.0       0
2004.0      10
2005.0       9
2006.0      12
2007.0      23
2008.0      25
2009.0      16
2010.0      62
2011.0     107
2012.0      90
2013.0     119
2014.0     800
2015.0     109
2016.0    1447
2017.0      91
2018.0     277
2019.0     104
2020.0     171
2021.0      51
Name: radius, dtype: int64


Получили Series, где по годам открытия расписано количество экзопланет, для которых удалось установить радиус.

Посмотрим, как меняется средний радиус открытых экзопланет год от года. Для этого надо сложить радиусы планет, открытых за определённый год, и поделить на их количество (которое мы уже нашли).

Сумма радиусов считается методом **sum()**:

**exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()**


**print(exo_radius_sum)**



In [ ]:
import pandas as pd
exoplanet = pd.read_csv('exoplanet_catalog.csv')
exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()

print(exo_radius_sum)

discovered
1988.0      0.000000
1992.0      0.000000
1995.0      1.900000
1996.0      1.060000
1998.0      0.000000
1999.0      2.410000
2000.0      5.107000
2001.0      0.921000
2002.0      4.210000
2003.0      0.000000
2004.0      9.909700
2005.0      9.999000
2006.0     22.425000
2007.0     29.088600
2008.0     34.329000
2009.0     16.416794
2010.0     62.760660
2011.0     86.187374
2012.0     56.441507
2013.0     80.399100
2014.0    194.348230
2015.0     66.196690
2016.0    403.726600
2017.0     59.428622
2018.0    130.962070
2019.0     70.835372
2020.0     85.252100
2021.0     43.260410
Name: radius, dtype: float64


Очень кстати, что объекты Series можно делить друг на друга. Это позволит нам разделить перечень сумм радиусов на перечень количеств экзопланет без перебора в цикле:


**exo_radius_mean = exo_radius_sum/exo_number**

**print(exo_radius_mean)**


In [ ]:
import pandas as pd
exoplanet = pd.read_csv('exoplanet_catalog.csv')
exo_radius_mean = exo_radius_sum/exo_number

print(exo_radius_mean)

discovered
1988.0         NaN
1992.0         NaN
1995.0    1.900000
1996.0    1.060000
1998.0         NaN
1999.0    1.205000
2000.0    1.021400
2001.0    0.921000
2002.0    1.052500
2003.0         NaN
2004.0    0.990970
2005.0    1.111000
2006.0    1.868750
2007.0    1.264722
2008.0    1.373160
2009.0    1.026050
2010.0    1.012269
2011.0    0.805489
2012.0    0.627128
2013.0    0.675623
2014.0    0.242935
2015.0    0.607309
2016.0    0.279009
2017.0    0.653062
2018.0    0.472787
2019.0    0.681109
2020.0    0.498550
2021.0    0.848243
Name: radius, dtype: float64


Точность наших приборов растёт, и новые экзопланеты по размерам всё ближе к Земле.

А теперь вернемся к анализу датафрейма с музыкльными предпочтениями слушателей Яндекс.Музыки. Идею объединения сервисов Музыка и Радио тестировали на небольшой группе пользователей. Результаты сведены в csv-файл, который вам предстоит изучить. Итог анализа таких данных — это метрики: величины, значения которых отражают пользовательские впечатления. Одна из важнейших — **happiness**. Здесь это среднее время, которое пользователь слушает музыку в течение выбранного периода времени (в нашей задаче — за сутки). Чем дольше пользователь слушает музыку, тем он довольнее. Ваша задача: найти значение happiness и посмотреть, как оно менялось.

Тем же методом **groupby()**, которым мы ищем новую Землю, можно поискать и необыкновенного человека в данных Яндекс.Музыки. Тем более, что без этого не выполнить поставленной менеджером задачи.

Прежде, чем рассчитывать метрику happiness, нужно изучить пользователей, чьё «счастье» мы собираемся оценить. Какие они, эти люди, которые слушают действительно много музыки? Есть ли у них особые предпочтения, или они потребляют всё подряд?


## Задача

1. Меломаны у нас есть. Сейчас узнаем идентификатор **user_id** одного из них. Для этого сгруппируем данные по каждому пользователю, чтобы собрать жанры прослушанных им композиций.

Сгруппируйте DataFrame по столбцу user_id, сохраните полученный результат в переменной genre_grouping.

Посчитайте количество жанров, которые выбрали пользователи, методом count(), указав, что выбираем один столбец genre_name.

Сохраните результат в переменной genre_counting и выведите первые 30 строк этой таблицы.

2. Быть может, те, кто за день слушает больше 50 песен, имеют более широкие предпочтения. Чтобы найти такого, изготовим универсальный инструмент.

Напишите функцию user_genres, которая принимает некую группировку как свой аргумент group. Функция должна перебирать группы, входящие в эту группировку.

В каждой группе два элемента — имя группы с индексом 0 и список значений с индексом 1.

Обнаружив такую группу, в которой список (элемент с индексом 1) содержит более 50 значений, функция возвращает имя группы (значение элемента с индексом 0).

3. Вызовите функцию **user_genres**, как аргумент передайте ей **genre_grouping**. Результат – **user_id** неведомого нам любителя музыки – сохраните в переменной **search_id** и выведите значение на экран.


In [ ]:
import pandas as pd
df = pd.read_csv('yandex_music (2).csv')
genre_grouping = df.groupby('  userID')

print(genre_grouping)

genre_counting= genre_grouping['genre'].count()

print(genre_counting.head(30))


def user_genres(group):
    for i in group:
        if len(i[1]) > 50:
            return i[0]


search_id = user_genres(genre_grouping)

print('User ID с более чем 50 прослушанными песнями:', search_id)




  userID
10004220    3
100173ED    1
1001CE90    7
1002410A    3
100261F9    2
1004AAE6    1
1004C5D8    1
1005BECA    1
10089377    3
100AB021    1
100ACFBC    1
10101201    2
10107FC7    2
10112F4C    2
1016144E    1
1016C39F    2
10180175    1
101861D1    2
10189D10    8
101AAA19    1
101B046A    2
101B53F1    1
101BBC61    1
101C12A4    2
101D714C    1
101E910F    1
10211433    1
10222F55    1
1022B77E    1
10246DCB    2
Name: genre, dtype: int64
User ID с более чем 50 прослушанными песнями: 414F229D


# Сортировка данных

Поиск необычного в группе — что среди планет, что среди меломанов — это прежде всего поиск чемпионов: объектов с выдающимися показателями по разным статьям. Как всю таблицу, так и отдельные группы изучают, сортируя строки по какому-либо столбцу.

В Pandas для этой операции есть метод **sort_values()**. У него два аргумента:

• **by = 'имя столбца'** — имя столбца, по которому нужно сортировать;

• **ascending:** по умолчанию True. Для сортировки по убыванию установите значение False.

![image.png](attachment:image.png)


Среди экзопланет интересны близкие по размерам к Земле. Есть ли такие? Отсортируем список по радиусу в порядке возрастания. Тогда в голове таблицы окажутся самые малые, на которых гравитация не прижмёт нас к полу.

**print(exoplanet.sort_values(by='radius').head(30))**


In [ ]:
import pandas as pd
exoplanet = pd.read_csv('exoplanet_catalog.csv')
print(exoplanet.sort_values(by='radius').head(30))


                 # name planet_status     mass  mass_error_min  \
9          1I/'Oumuamua     Confirmed      NaN             NaN   
16           2I/Borisov     Confirmed      NaN             NaN   
4357  SDSS J1228+1040 b     Confirmed      NaN             NaN   
3258       Kepler-391 b     Confirmed      NaN             NaN   
3207        Kepler-37 b     Confirmed  0.01000         0.01000   
3354       Kepler-444 b     Confirmed      NaN             NaN   
1744       Kepler-102 b     Confirmed  0.01353         0.01353   
3355       Kepler-444 c     Confirmed      NaN             NaN   
2126      Kepler-1308 b     Confirmed      NaN             NaN   
2221       Kepler-138 b     Confirmed  0.00021         0.00012   
3356       Kepler-444 d     Confirmed      NaN             NaN   
3611        Kepler-62 c     Confirmed  0.01300             NaN   
3357       Kepler-444 e     Confirmed      NaN             NaN   
3323        Kepler-42 d     Confirmed  0.00300             inf   
1745      

Оказывается, некоторые из уже открытых экзопланет по размерам близки не то что к Земле, но уже и к Луне! Получим список экзопланет с радиусом меньше земного. Смотрите, как логический оператор (здесь это <) задействуется в отборе элементов столбца. Дальше нам этот приём не раз понадобится.


**print(exoplanet[exoplanet['radius'] < 1])**

In [ ]:
import pandas as pd
exoplanet = pd.read_csv('exoplanet_catalog.csv')
print(exoplanet[exoplanet['radius'] < 1])

                # name planet_status      mass  mass_error_min  \
9         1I/'Oumuamua     Confirmed       NaN             NaN   
16          2I/Borisov     Confirmed       NaN             NaN   
22        2M 2140+16 b     Confirmed  20.00000        20.00000   
41    2MASS J0348-6022     Confirmed  43.00000        18.00000   
42    2MASS J0407+1546     Confirmed  67.00000        28.00000   
...                ...           ...       ...             ...   
4731        Wolf 503 b     Confirmed       NaN             NaN   
4732        Wolf 940 b     Confirmed  26.00000         6.00000   
4734           XO-2N b     Confirmed   0.62000         0.02000   
4751  ZTF J0038+2030 b     Confirmed  59.30000         3.90000   
4783          pi Men c     Confirmed   0.01517         0.00267   

      mass_error_max  mass_sini  mass_sini_error_min  mass_sini_error_max  \
9                NaN        NaN                  NaN                  NaN   
16               NaN        NaN                  NaN 

Но и этот список такой длинный, что изучать его лучше по частям. Экзопланеты, близкие по размерам к Земле, за последнее десятилетие открывали нередко. Можно изучать список открытых за каждый год. Например, для 2014 года (вновь обратите внимание на работу логического оператора, теперь это ==):

**print(exoplanet[exoplanet['discovered'] == 2014])**


In [ ]:
import pandas as pd
exoplanet = pd.read_csv('exoplanet_catalog.csv')
print(exoplanet[exoplanet['discovered'] == 2014])

            # name planet_status  mass  mass_error_min  mass_error_max  \
106   BD-06 1339 d     Confirmed   NaN             NaN             NaN   
222       GJ 15A b     Confirmed   NaN             NaN             NaN   
223     GJ 160.2 b     Confirmed   NaN             NaN             NaN   
233       GJ 180 b     Confirmed   NaN             NaN             NaN   
234       GJ 180 c     Confirmed   NaN             NaN             NaN   
...            ...           ...   ...             ...             ...   
4763     eta Cet b     Confirmed   NaN             NaN             NaN   
4764     eta Cet c     Confirmed   NaN             NaN             NaN   
4776      mu Leo b     Confirmed   NaN             NaN             NaN   
4784     sig Per b     Confirmed   NaN             NaN             NaN   
4795     zet Del B     Confirmed  40.0             5.0            15.0   

      mass_sini  mass_sini_error_min  mass_sini_error_max  radius  \
106    0.058200             0.029600      

А чтобы не тратить время на лишнее, поставим оба условия сразу. Для этого в Pandas есть логический оператор &, подобный оператору and языка Python. Напомним, его смысл на русском языке можно передать словами «и ещё»:

**# экзопланеты меньше Земли __ и ещё __ открытые в 2014 году**

**exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]**

**print(exo_small_14)**



In [ ]:
import pandas as pd
exoplanet = pd.read_csv('exoplanet_catalog.csv')
# экзопланеты меньше Земли __ и ещё __ открытые в 2014 году

exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]

print(exo_small_14)

            # name planet_status    mass  mass_error_min  mass_error_max  \
400     HAT-P-54 b     Confirmed  0.7600         0.03200         0.03200   
465       HATS-5 b     Confirmed  0.2370         0.01200         0.01200   
477       HATS-6 b     Confirmed  0.3190         0.07000         0.07000   
1164  HIP 116454 b     Confirmed  0.0372         0.00419         0.00419   
1690    KOI-1257 b     Confirmed  1.4500         0.35000         0.35000   
...            ...           ...     ...             ...             ...   
4039   Kepler-95 b     Confirmed  0.0410         0.00900         0.00900   
4051   Kepler-96 b     Confirmed  0.0270         0.01100         0.01100   
4065   Kepler-97 b     Confirmed  0.0110         0.00600         0.00600   
4077   Kepler-98 b     Confirmed  0.0110         0.00500         0.00500   
4088   Kepler-99 b     Confirmed  0.0190         0.00400         0.00400   

      mass_sini  mass_sini_error_min  mass_sini_error_max  radius  \
400         NaN   

Отсортируем результат в порядке убывания радиуса.

**print(exo_small_14.sort_values(by = 'radius', ascending = False))**


In [ ]:
import pandas as pd
exoplanet = pd.read_csv('exoplanet_catalog.csv')
print(exo_small_14.sort_values(by = 'radius', ascending = False))

import pandas as pd
exoplanet = pd.read_csv('exoplanet_catalog.csv')
exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]
print(exo_small_14.sort_values(by = 'radius', ascending = False))

            # name planet_status     mass  mass_error_min  mass_error_max  \
477       HATS-6 b     Confirmed  0.31900         0.07000         0.07000   
3329  Kepler-425 b     Confirmed  0.25000         0.08000         0.08000   
3319  Kepler-419 b     Confirmed  2.50000         0.30000         0.30000   
400     HAT-P-54 b     Confirmed  0.76000         0.03200         0.03200   
1690    KOI-1257 b     Confirmed  1.45000         0.35000         0.35000   
...            ...           ...      ...             ...             ...   
2263  Kepler-141 b     Confirmed      NaN             NaN             NaN   
1745  Kepler-102 c     Confirmed  0.00944         0.00944         0.00000   
2221  Kepler-138 b     Confirmed  0.00021         0.00012         0.00019   
1744  Kepler-102 b     Confirmed  0.01353         0.01353         0.00000   
3258  Kepler-391 b     Confirmed      NaN             NaN             NaN   

      mass_sini  mass_sini_error_min  mass_sini_error_max   radius  \
477  

Самая крупная планета, Kepler 106 d – почти как Земля, вращается вокруг звезды Kepler 106 в созвездии Лебедя. Эта звезда очень похожа на наше Солнце. Правда, до неё 1435 световых лет — далековато. Но, возможно, учёные что-нибудь придумают. А мы пока применим эту технологию к нашему бизнесу, в «приземлённой» задаче.
## Задача

1. Космический телескоп Kepler открыл похожую на Землю планету у похожей на Солнце звезды. А вы в данных Яндекс.Музыки обнаружили меломана с уникальными данными. Он за день послушал больше 50 композиций.
Получите таблицу с прослушанными им треками.

Для этого запросите из структуры данных df строки, отвечающие сразу двум условиям:

1) значение в столбце 'user_id' должно быть равно значению переменной search_id;

2) время прослушивания, т.е. значение в столбце 'total_play_seconds', не должно равняться 0.

Сохраните результат в переменной music_user.

2. Теперь узнаем, сколько времени он слушал музыку каждого жанра.

Сгруппируйте данные таблицы music_user по столбцу 'genre_name' и получите сумму значений столбца 'total_play_seconds'. Сохраните результат в переменной sum_music_user и выведите её значение на экран.

3. Кажется, предпочтения нашего меломана начинают проявляться. Но, возможно, длительность композиций от жанра к жанру сильно различается. Важно знать, сколько треков каждого жанра он включил.

Сгруппируйте данные по столбцу genre_name и посчитайте, сколько значений в столбце genre_name. Сохраните результат в переменной count_music_user и выведите её значение на экран.

4. Чтобы предпочтения были видны сразу, нужно крупнейшие значения расположить наверху. Отсортируйте данные в группировке sum_music_user по убыванию. Внимание: когда применяете метод sort_values() к Series с единственным столбцом, аргумент by указывать не нужно, только порядок сортировки.

Сохраните результат в переменной final_sum и выведите её значение на экран.

5. Теперь то же самое надо сделать с числом прослушанных меломаном композиций. Отсортируйте данные группировки count_music_user по убыванию. Сохраните результат в переменной final_count, значение которой выведите на экран.


In [ ]:
import pandas as pd
df = pd.read_csv('yandex_music (2).csv')

search_id = '55204538'
music_user = df[(df['  userID'] == search_id) & (df['time'] != 0)]

sum_music_user = music_user.groupby('genre')['time'].sum()
print('Время прослушивания по жанрам:')
print(sum_music_user)

count_music_user = music_user.groupby('genre')['genre'].count()
print('Количество прослушанных треков по жанрам:')
print(count_music_user)

final_sum = sum_music_user.sort_values(ascending=False)
print('Сортировка по времени прослушивания:')
print(final_sum)

final_count = count_music_user.sort_values(ascending=False)
print('Сортировка по количеству прослушанных треков:')
print(final_count)

Время прослушивания по жанрам:
genre
postrock    14:47:16
rock        14:07:09
Name: time, dtype: object
Количество прослушанных треков по жанрам:
genre
postrock    1
rock        1
Name: genre, dtype: int64
Сортировка по времени прослушивания:
genre
postrock    14:47:16
rock        14:07:09
Name: time, dtype: object
Сортировка по количеству прослушанных треков:
genre
postrock    1
rock        1
Name: genre, dtype: int64
